In [1]:
# Sheet properties

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [3]:
# Imports

In [4]:
import sys
sys.path.append('/home/rcendre/classification')
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  
from numpy import array
from sklearn.preprocessing import LabelEncoder
from toolbox.models.builtin import Applications
from toolbox.classification.common import IO, Tools, Folds
from toolbox.classification.parameters import Settings, Dermatology
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import Views, ViewsTools
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Parameters

In [6]:
validation = 4
settings = Settings.get_default_dermatology()
max_iter = -1

In [7]:
# Inputs

In [8]:
inputs_patch = Dermatology.images(modality='Microscopy', data_type='Patch', use_unknown=False)
inputs_full = Dermatology.images(modality='Microscopy', data_type='Full', use_unknown=False)
prediction_file = f'Prediction_Curriculum.pickle'

In [9]:
# Transform groups
group_encoder = LabelEncoder().fit(array(inputs_full['ID_Patient'].tolist()))
Tools.transform(inputs_full, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
Tools.transform(inputs_patch, {'datum': 'Label'}, label_encoder, 'LabelEncode')
Tools.transform(inputs_full, {'datum': 'Label'}, label_encoder, 'LabelEncode')
# Make folds
Folds.build_group_folds(inputs_full, {'datum': 'Datum', 'label_encode': 'LabelEncode', 'group': 'GroupEncode'}, validation)

In [10]:
# Models

In [11]:
additionnal = {'batch_size': 8,
            'epochs' : 15,
            'shuffle': True,
            'rotation_range': 45,
            'width_shift_range': 0.1,
            'height_shift_range': 0.1,
            'horizontal_flip': True,
            'vertical_flip': True,
            'fill_mode': 'wrap'}
model = Applications.get_fine_tuning(3, 0, -1, architecture='ResNet', pooling='max', activation='softmax', additional=additionnal)

In [ ]:
# Evaluation

In [ ]:
# model_pretrained = Tools.fit(inputs_patch, {'datum': 'Datum', 'label_encode': 'LabelEncode'}, model)
# model_pretrained.save('pretrained')

In [13]:
model.load('pretrained')
Tools.evaluate(inputs_full, {'datum': 'Datum', 'label_encode': 'LabelEncode'}, model, 'Curriculum')

# Save
IO.save(inputs_full, prediction_file)

Found 4073 images belonging to 3 classes.
Pre-training...
  ...
    to  
  ['...']
Train for 510 steps
Epoch 1/15
510/510 [==============================] - 872s 2s/step - loss: 2.0785 - accuracy: 0.7383
Epoch 2/15
510/510 [==============================] - 867s 2s/step - loss: 1.7884 - accuracy: 0.7663
Epoch 3/15
510/510 [==============================] - 868s 2s/step - loss: 1.6055 - accuracy: 0.7871
Epoch 4/15
510/510 [==============================] - 867s 2s/step - loss: 1.4219 - accuracy: 0.7940
Epoch 5/15
510/510 [==============================] - 866s 2s/step - loss: 1.4899 - accuracy: 0.8026
Epoch 6/15
510/510 [==============================] - 864s 2s/step - loss: 1.2286 - accuracy: 0.8198
Epoch 7/15
510/510 [==============================] - 867s 2s/step - loss: 1.2112 - accuracy: 0.8213
Epoch 8/15
510/510 [==============================] - 869s 2s/step - loss: 1.2444 - accuracy: 0.8193
Epoch 9/15
510/510 [==============================] - 869s 2s/step - loss: 1.3589 - accur

510/510 [==============================] - 883s 2s/step - loss: 1.4231 - accuracy: 0.8011
Epoch 12/15
510/510 [==============================] - 875s 2s/step - loss: 1.5414 - accuracy: 0.8006
Epoch 13/15
510/510 [==============================] - 872s 2s/step - loss: 1.1132 - accuracy: 0.8269
Epoch 14/15
510/510 [==============================] - 879s 2s/step - loss: 1.3020 - accuracy: 0.8090
Epoch 15/15
510/510 [==============================] - 877s 2s/step - loss: 1.2028 - accuracy: 0.8178
Final-training...
  ...
    to  
  ['...']
Train for 510 steps
Epoch 1/15
510/510 [==============================] - 894s 2s/step - loss: 2.2802 - accuracy: 0.7778
Epoch 2/15
510/510 [==============================] - 890s 2s/step - loss: 1.7734 - accuracy: 0.7997
Epoch 3/15
510/510 [==============================] - 882s 2s/step - loss: 0.9979 - accuracy: 0.8357
Epoch 4/15
510/510 [==============================] - 878s 2s/step - loss: 1.0577 - accuracy: 0.8362
Epoch 5/15
510/510 [===============

In [ ]:
# Scores and ROC

In [14]:
from IPython.display import HTML
from IPython.display import display

inputs = IO.load(prediction_file)

# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))

# Label
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LabelEncode', 'eval': 'Curriculum'}, label_encoder)],
                                                                title=[f'Test - Curriculum'])))

,precision,recall,f1-score,support
Normal,0.21±0.10,0.89±0.06,0.34±0.14,757.00±115.20
Benign,0.89±0.23,0.25±0.12,0.38±0.17,2204.00±115.21
Malignant,0.86±0.06,0.55±0.08,0.67±0.05,2470.00±125.50
accuracy,0.47±0.04,0.47±0.04,0.47±0.04,0.47±0.04
macro avg,0.65±0.08,0.56±0.04,0.46±0.05,5431.00±0.43
weighted avg,0.78±0.14,0.47±0.04,0.51±0.07,5431.00±0.43
